<a href="https://colab.research.google.com/github/PeterHJY628/MyOwnExample/blob/main/%E2%80%9CLoRA_PitVQA_Sentence_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Download code
!git clone https://github.com/HRL-Mike/PitVQA.git

#Download Dataset
!mkdir /content/PitVQA/datasets
%cd /content/PitVQA/datasets
!gdown --id 1FoAEY_u0PTAlrscjEifi2om15A83wL78

# Unzipping the VQA EndoVis18 Dataset
!unzip -q EndoVis-18-VQA.zip
%cd /content/PitVQA

Cloning into 'PitVQA'...
remote: Enumerating objects: 401, done.
remote: Counting objects: 100% (139/139), done.
remote: Compressing objects: 100% (138/138), done.
remote: Total 401 (delta 74), reused 0 (delta 0), pack-reused 262 (from 1)
Receiving objects: 100% (401/401), 14.44 MiB | 15.45 MiB/s, done.
Resolving deltas: 100% (199/199), done.
/content/PitVQA/datasets
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1FoAEY_u0PTAlrscjEifi2om15A83wL78
From (redirected): https://drive.google.com/uc?id=1FoAEY_u0PTAlrscjEifi2om15A83wL78&confirm=t&uuid=a941161d-63d0-478a-a703-abb14d6ba6c8
To: /content/PitVQA/datasets/EndoVis-18-VQA.zip
100% 2.71G/2.71G [00:59<00:00, 45.5MB/s]
/content/PitVQA


In [2]:
!pip install -q timm==0.9.12 fairscale==0.4.13 scikit-learn==1.3.2
#instal python package

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 23.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 117.5 MB/s eta 0:00:00


In [3]:
!pip -q install peft

In [4]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import GPT2Model

gpt = GPT2LMHeadModel.from_pretrained('gpt2')
gpt

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [5]:
from transformers import AutoModelForSeq2SeqLM #transformers is the class lib from hugging face
from peft import LoraModel, LoraConfig, get_peft_model #Parameter-Efficient Fine-Tuning class

config = LoraConfig(
    # task_type="CAUSAL_LM",
    task_type="SEQ_2_SEQ_LM",
    r=8,
    lora_alpha=32,
    # target_modules=["q", "v"],
    # target_modules=["wte"],
    target_modules=["c_attn"],#self attention module
    # target_modules=["wte"],
    lora_dropout=0.01,
)

lora_model = get_peft_model(gpt, config)#combine oretrained model with LoRA config
lora_model

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2SdpaAttention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D(nf=2304, nx=768)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.01, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
     

### Dataloader

In [6]:
import os
import glob

from PIL import Image#load and process image
from torch.utils.data import Dataset# PyTorch 数据集类的基类，用于创建自定义数据集。
import torchvision.transforms as transforms#提供了一系列图像预处理和数据增强工具，例如图像缩放、裁剪、归一化等。
from torch.utils.data import DataLoader#PyTorch 的数据加载器，可以高效地加载批量数据，支持多线程。
from pathlib import Path#用于跨平台操作文件路径，更现代和灵活。
from torchvision.transforms.functional import InterpolationMode#提供的插值模式，用于图像缩放时的插值方法，例如最近邻插值、双线性插值等。

class EndoVis18VQAGPTGen(Dataset):
    def __init__(self, seq, folder_head, folder_tail):

        self.transform = transforms.Compose([
            transforms.Resize((224, 224), interpolation=InterpolationMode.BICUBIC),  # input image size
            transforms.ToTensor(),#将图像转换为 PyTorch 张量，归一化到 [0, 1] 范围
        ])

        # files, question and answers
        filenames = []
        for curr_seq in seq:
            filenames = filenames + glob.glob(folder_head + str(curr_seq) + folder_tail)
        self.vqas = []
        for file in filenames:
            file_data = open(file, "r")
            lines = [line.strip("\n") for line in file_data if line != "\n"]
            file_data.close()
            for line in lines:
                self.vqas.append([file, line])
        print('Total files: %d | Total question: %.d' % (len(filenames), len(self.vqas)))

        # Labels
        self.labels = ['kidney',
                'Idle', 'Grasping', 'Retraction', 'Tissue_Manipulation',
                'Tool_Manipulation', 'Cutting', 'Cauterization', 'Suction',
                'Looping', 'Suturing', 'Clipping', 'Staple', 'Ultrasound_Sensing',
                'left-top', 'right-top', 'left-bottom', 'right-bottom'] #18 labels

    def __len__(self):
        return len(self.vqas)

    def __getitem__(self, idx):
        qa_full_path = Path(self.vqas[idx][0])
        seq_path = qa_full_path.parents[2]
        file_name = self.vqas[idx][0].split('/')[-1]  # / in linux and \\ in windows

        # img
        img_loc = os.path.join(seq_path, 'left_fr', file_name.split('_')[0] + '.png')
        raw_image = Image.open(img_loc).convert('RGB')
        img = self.transform(raw_image)#self.transform：对图像应用预处理，例如调整大小、转换为张量等。

        # question and answer
        question, answer = self.vqas[idx][1].split('|')

        return img_loc, img, question, answer

### Model

In [7]:
import torch
from torch import nn

from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import ViTModel, BlipConfig, BlipTextModel

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#LoRA module class,用于调整自注意力机制中的 qkv 权重。
class _LoRA_qkv(nn.Module):
    def __init__(self, w_qkv, w_a, w_b, lora_alpha, lora_dropout):
        super().__init__()
        self.w_qkv = w_qkv
        self.w_a = w_a #Project the input down to a lower-dimension space
        self.w_b = w_b #Project the data in the lower-dimension space back to the original dimension
        self.lora_alpha = lora_alpha #Scaling factor for the LoRA adjustments. It determines how much influence the LoRA-modified weights have.
        self.dropout = nn.Dropout(lora_dropout) #Dropout rate
        self.scaling = self.lora_alpha / self.w_a.weight.shape[0]  # alpha / r

        self.weight = self.w_qkv.weight  # load original weights

    def forward(self, x):
        return self.w_qkv(x) + self.scaling * self.dropout(self.w_b(self.w_a(x)))

class LoRAInitializer:
    def __init__(self, model, r=None, lora=None, lora_alpha=32, lora_dropout=0.1):
        if r is None:
            r = [14, 14, 12, 12, 10, 10, 8, 8, 8, 8, 8, 8]#Prepared for 12 blocks of QKV blocks
        if lora is None:
            lora = ['q', 'v']

        self.model = model
        self.r = r
        self.lora = lora
        self.lora_alpha = lora_alpha
        self.lora_dropout = lora_dropout
        #ab matrix
        self.w_As = []
        self.w_Bs = []

    def reset_parameters(self):
        for w_A, w_B in zip(self.w_As, self.w_Bs):
            # normal distribution init for w_A
            nn.init.normal_(w_A.weight, mean=0.0, std=0.02)
            nn.init.zeros_(w_B.weight)  # zero init for w_B
    '''
    将 Transformer 模型的 QKV 权重替换为带有 LoRA 的 _LoRA_qkv 模块。
    冻结 Transformer 模型中的所有原始权重，使得在训练过程中仅 LoRA 参数（w_a 和 w_b）可训练
    '''

    def initialize_lora(self):
        for param in self.model.transformer.parameters():
            param.requires_grad = False  # freeze transformer parameters
            # param.requires_grad = True

        for t_layer_i, blk in enumerate(self.model.transformer.h):  # t_layer_i = [0, 11], blk = transformer block
            # GPT2 uses a single c_attn for q, k, v
            w_qkv = blk.attn.c_attn
            in_features = w_qkv.weight.shape[0]  # 768
            out_features = w_qkv.weight.shape[1]  # 2304

            w_a_linear = nn.Linear(in_features, self.r[t_layer_i], bias=False)
            w_b_linear = nn.Linear(self.r[t_layer_i], out_features, bias=False)
            self.w_As.append(w_a_linear)
            self.w_Bs.append(w_b_linear)
            blk.attn.c_attn = _LoRA_qkv(w_qkv, w_a_linear, w_b_linear, self.lora_alpha, self.lora_dropout)

        self.reset_parameters()
        print("LoRA params initialized!")
        return self.model


class BLIPGPTVQAGen(nn.Module):
    def __init__(self, r=None, lora=None, lora_alpha=32, lora_dropout=0.1):
        super(BLIPGPTVQAGen, self).__init__()

        # gpt2 decoder
        self.gpt = GPT2LMHeadModel.from_pretrained('gpt2')
        self.gpt = LoRAInitializer(self.gpt, r=r, lora=lora, lora_alpha=lora_alpha,
                       lora_dropout=lora_dropout).initialize_lora()  # add lora

        # visual encoder
        model_name = "google/vit-base-patch16-224-in21k"
        self.visual_encoder = ViTModel.from_pretrained(model_name)

        # tokenizer-GPT
        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        self.tokenizer.pad_token = self.tokenizer.eos_token  # end of string

        # text encoder
        config = BlipConfig.from_pretrained("Salesforce/blip-vqa-base")
        self.text_encoder = BlipTextModel(config.text_config, add_pooling_layer=False)

        # modify embedding layer
        new_vocab_size = len(self.tokenizer)
        embedding_dim = self.text_encoder.embeddings.word_embeddings.embedding_dim
        self.text_encoder.embeddings.word_embeddings = nn.Embedding(new_vocab_size, embedding_dim)  # He init

    def forward(self, image, question_inputs):
        # visual encoder
        image = image.to(device)
        image_embeds = self.visual_encoder(image).last_hidden_state  # torch.Size([bs, 197, 768]) on default num_patches = 196(14*14), here +1 class toke; 768 is hidden dimension
        image_atts = torch.ones(image_embeds.size()[:-1], dtype=torch.long).to(image.device)  # torch.Size([bs, 197])# attention mask 196,not 197?

        question_input_ids = question_inputs['input_ids']  # torch.Size([bs, 25])25 is the longest length of the text
        question_att_mask = question_inputs['attention_mask']

        # multimodal encoder
        text_output = self.text_encoder(input_ids=question_input_ids,
                         attention_mask=question_att_mask,
                         encoder_hidden_states=image_embeds,
                         encoder_attention_mask=image_atts,
                         return_dict=True)
        text_embeds = text_output.last_hidden_state  # torch.Size([bs, 25, 768]), args.question_len=25

        # text decoder
        gpt_output = self.gpt(inputs_embeds=text_embeds,
                              encoder_attention_mask=question_att_mask)  # torch.Size([bs, 25, 50257])
        return gpt_output.logits

### Utils

In [8]:
def adjust_learning_rate(optimizer, shrink_factor):
    print("\nDECAYING learning rate.")
    for param_group in optimizer.param_groups:
        param_group['lr'] = param_group['lr'] * shrink_factor
    print("The new learning rate is %f\n" % (optimizer.param_groups[0]['lr'],))

class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

### Main

In [9]:
import os
import torch
import argparse
import torch.utils.data
import numpy as np
import random

from torch import nn
from torch.utils.data import DataLoader
from transformers import GPT2Tokenizer

from nltk.translate.bleu_score import corpus_bleu
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

import json
from google.colab import drive

# 挂载 Google Drive
drive.mount('/content/drive')
drive_path = '/content/drive/My Drive/Share_Weights'
os.makedirs(drive_path, exist_ok=True)

def save_checkpoint(model, optimizer, epoch, metrics, file_name="Lora_checkpoint.pt"):
    """保存模型和训练状态到 Google Drive"""
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'metrics': metrics,
        'best_epoch': best_epoch,
    }
    save_path = os.path.join(drive_path, file_name)
    torch.save(checkpoint, save_path)
    print(f"Checkpoint saved to {save_path}")

def load_checkpoint(file_name="checkpoint.pt"):
    """从 Google Drive 加载模型和训练状态"""
    load_path = os.path.join(drive_path, file_name)
    if os.path.exists(load_path):
        checkpoint = torch.load(load_path)
        print(f"Checkpoint loaded from {load_path}")
        return checkpoint
    else:
        print("No checkpoint found, starting from scratch.")
        return None

def save_metrics_to_drive(metrics, file_name="metrics.json"):
    """保存评估结果到 Google Drive"""
    save_path = os.path.join(drive_path, file_name)
    with open(save_path, 'w') as f:
        json.dump(metrics, f, indent=4)
    print(f"Evaluation metrics saved to {save_path}")

def train(args, train_dataloader, model, criterion, optimizer, epoch, tokenizer, device):
    model.train()
    total_loss = AverageMeter()

    for i, (_, images, questions, answers) in enumerate(tqdm(train_dataloader), 0):
        question_inputs = tokenizer(questions, padding="max_length", max_length=int(args.seq_length),
                                    return_tensors="pt", truncation=True)#truncation=True：截断超过最大长度的序列。 return_tensors="pt"：返回 PyTorch 张量
        answer_inputs = tokenizer(answers, padding="max_length", max_length=int(args.seq_length),
                                  return_tensors="pt", truncation=True)

        # get logits and labels
        logits = model(image=images.to(device), question_inputs=question_inputs.to(device))
        # 文本生成模型的输出代表模型对每个位置的下一个词的预测
        labels = answer_inputs['input_ids'].to(device)

        # get shifted logits and labels
        shift_logits = logits[:, :-1, :].contiguous()
        shift_labels = labels[:, 1:].contiguous()

        # compute loss
        loss = criterion(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss.update(loss.item())
    print("Epoch: {}/{} Loss: {:.6f} AVG_Loss: {:.6f}".format(epoch, args.epochs, total_loss.val, total_loss.avg))

def validate(args, val_loader, model, criterion, epoch, tokenizer, device):
    references = []
    hypotheses = []

    model.eval()
    total_loss = AverageMeter()
    with torch.no_grad():
        for i, (_, images, questions, answers) in enumerate(tqdm(val_loader), 0):
            question_inputs = tokenizer(questions, padding="max_length", max_length=int(args.seq_length),
                                        return_tensors="pt", truncation=True)
            answer_inputs = tokenizer(answers, padding="max_length", max_length=int(args.seq_length),
                                      return_tensors="pt", truncation=True)

            # get logits and labels
            logits = model(image=images.to(device), question_inputs=question_inputs.to(device))
            labels = answer_inputs['input_ids'].to(device)

            # get shifted logits and labels
            shift_logits = logits[:, :-1, :].contiguous()
            shift_labels = labels[:, 1:].contiguous()

            # compute loss
            loss = criterion(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            total_loss.update(loss.item())

            # generate predicted answer
            _, predicted = torch.max(logits, dim=-1)

            # decode references and predictions
            reference_answers = tokenizer.batch_decode(labels, skip_special_tokens=True)
            predicted_answers = tokenizer.batch_decode(predicted, skip_special_tokens=True)

            # add references and hypotheses to lists
            for ref, hyp in zip(reference_answers, predicted_answers):
                references.append([ref.split()])
                hypotheses.append(hyp.split())

        # Calculate BLEU_1~4
        metrics = {}
        metrics["Bleu_1"] = corpus_bleu(references, hypotheses, weights=(1.00, 0.00, 0.00, 0.00))
        metrics["Bleu_2"] = corpus_bleu(references, hypotheses, weights=(0.50, 0.50, 0.00, 0.00))
        metrics["Bleu_3"] = corpus_bleu(references, hypotheses, weights=(0.33, 0.33, 0.33, 0.00))
        metrics["Bleu_4"] = corpus_bleu(references, hypotheses, weights=(0.25, 0.25, 0.25, 0.25))
        print(f"Epoch: {epoch}/{args.epochs} EVA LOSS: {total_loss.avg:.6f} "
              f"BLEU-1: {metrics['Bleu_1']:.6f} BLEU-2: {metrics['Bleu_2']:.6f} "
              f"BLEU-3: {metrics['Bleu_3']:.6f} BLEU-4: {metrics['Bleu_4']:.6f}")
    return metrics

def seed_everything(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

class Args:
    def __init__(self):
        self.epochs = 20
        self.batch_size = 8
        self.workers = 8
        self.random_seed = 42
        self.seq_length = 32
        self.lr = 0.00002

        self.vector_rank = [14, 14, 12, 12, 10, 10, 8, 8, 8, 8, 8, 8]
        self.lora_alpha = 32
        self.lora_dropout = 0.1



Mounted at /content/drive


In [ ]:
if __name__ == '__main__':
    args = Args()
    os.makedirs('./checkpoints/', exist_ok=True)

    seed_everything(args.random_seed)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    start_epoch = 1
    best_epoch = [0]
    best_results = [0.0]
    epochs_since_improvement = 0

    # data location
    train_seq = [2, 3, 4, 6, 7, 9, 10, 11, 12, 14, 15]
    val_seq = [1, 5, 16]

    folder_head = '/content/PitVQA/datasets/EndoVis-18-VQA/seq_'
    folder_tail = '/vqa/Sentence/*.txt'

    # dataloader
    train_dataset = EndoVis18VQAGPTGen(train_seq, folder_head, folder_tail)
    train_dataloader = DataLoader(dataset=train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=2)
    val_dataset = EndoVis18VQAGPTGen(val_seq, folder_head, folder_tail)
    val_dataloader = DataLoader(dataset=val_dataset, batch_size=args.batch_size, shuffle=False, num_workers=2)

    print(f'num of elements: {len(args.vector_rank)}')
    model = BLIPGPTVQAGen(r=args.vector_rank, lora_alpha=args.lora_alpha, lora_dropout=args.lora_dropout)
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)  # same learning rate for LoRA weights and other weights

    model = model.to(device)
    pytorch_total_params = sum(p.numel() for p in model.parameters())
    print('model params: ', pytorch_total_params)
    criterion = nn.CrossEntropyLoss().to(device)

    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    tokenizer.pad_token = tokenizer.eos_token

    #read check point
    checkpoint = load_checkpoint()
    if checkpoint:
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        best_results = checkpoint['metrics']
        best_epoch = checkpoint['best_epoch']
        print(f"Resuming training from epoch {start_epoch} with best BLEU-4: {best_results['Bleu_4']} at epoch {best_epoch}")
    else:
        start_epoch = 1
        best_results = {"Bleu_4": 0.0}
        best_epoch = 0


    print('Start training.')
    for epoch in range(start_epoch, args.epochs+1):

        if epochs_since_improvement > 0 and epochs_since_improvement % 5 == 0:
            adjust_learning_rate(optimizer, 0.8)

        # train
        train(args, train_dataloader=train_dataloader, model=model, criterion=criterion, optimizer=optimizer,
              epoch=epoch, tokenizer=tokenizer, device=device)
        # validation
        metrics = validate(args, val_loader=val_dataloader, model=model, criterion=criterion, epoch=epoch,
                           tokenizer=tokenizer, device=device)

        if metrics["Bleu_4"] >= best_results[0]:
            epochs_since_improvement = 0
            best_results[0] = metrics["Bleu_4"]
            best_epoch[0] = epoch
            print(f'Best epoch: {epoch}, Best Bleu_4: {metrics["Bleu_4"]}')
        else:
            epochs_since_improvement += 1
            print("\nEpochs since last improvement: %d\n" % (epochs_since_improvement,))

        save_checkpoint(model, optimizer, epoch, metrics, best_epoch, file_name="checkpoint.pt")
        save_metrics_to_drive(metrics, file_name="metrics.json")
    print('End training.')

Total files: 1560 | Total question: 10574
Total files: 447 | Total question: 3216
num of elements: 12
LoRA params initialized!


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model params:  363611136
No checkpoint found, starting from scratch.
Start training.


  1%|          | 13/1322 [00:09<12:43,  1.72it/s]

In [ ]:
import torch

# 定义保存路径
save_path = '/content/drive/MyDrive/checkpoints/manual_checkpoint.pth'

# 保存当前状态
state = {
    'epoch': current_epoch,  # 替换为当前 epoch
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}
torch.save(state, save_path)
print(f"Checkpoint saved at {save_path}")


NameError: name 'current_epoch' is not defined